In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("new_cci_cleaned_data (1).csv")

In [3]:
df.head()

,Utterance,Sentiment,Topic,Topic_Name,fallback_type,text_len,word_count,avg_word_len,is_negative,is_positive,...,is_request,sentiment_conflict,frustration_count,anger_count,confusion_count,gratitude_count,severity,is_urgent,intent_macro,emotion_macro
0,Can I get assistance in LHR when I land on my ...,positive,2,Reservations and Ticketing,no fallback,174,33,5.272727,0,1,...,1,0,0,0,0,0,2.000,0,Complaint,Mixed
1,"If I book my flight through princess cruise, s...",positive,2,Reservations and Ticketing,no fallback,113,23,4.913043,0,1,...,1,0,0,0,0,0,0.000,0,Request,Mixed
2,"Okay,arrive in Oslo at 5:00pm and catch Flight...",positive,8,Special Services and Assistance,no fallback,114,24,4.750000,0,1,...,0,0,0,0,0,0,0.625,0,Information Query,Mixed
3,I did not see my canceled reservation in my ac...,negative,2,Reservations and Ticketing,no fallback,76,13,5.846154,1,0,...,0,0,0,0,0,0,2.000,0,Complaint,Mixed
4,I already booked a flight feb. 16th that i wou...,positive,2,Reservations and Ticketing,no fallback,76,16,4.750000,0,1,...,0,0,0,0,0,0,0.000,0,General,Mixed


In [5]:
df["Topic_Name"].value_counts()

Topic_Name
General Customer Service           6586
Loyalty and Rewards Program        4230
Reservations and Ticketing         2178
Billing and Refunds                 602
In-flight Experience                594
Baggage and Lost Items              568
Account and Technical Support       426
Special Services and Assistance     346
Flight Operations and Status         68
Name: count, dtype: int64

In [7]:
df["exclamation_count"].value_counts()

exclamation_count
0    15282
1      204
2       36
3       36
4       16
9       12
5       12
Name: count, dtype: int64

In [15]:
df["gratitude_count"].value_counts()

gratitude_count
0    15188
1      410
Name: count, dtype: int64

In [14]:
df["severity"].min()

np.float64(0.0)

In [11]:
df.isnull().sum()

Utterance             0
Sentiment             0
Topic                 0
Topic_Name            0
fallback_type         0
text_len              0
word_count            0
avg_word_len          0
is_negative           0
is_positive           0
is_neutral            0
clean_text            0
question_count        0
exclamation_count     0
punct_density         0
tb_polarity           0
tb_subjectivity       0
readability_score     0
is_complaint          0
is_request            0
sentiment_conflict    0
frustration_count     0
anger_count           0
confusion_count       0
gratitude_count       0
severity              0
is_urgent             0
intent_macro          0
emotion_macro         0
dtype: int64

In [16]:
# NEGATIVE + QUESTION LOGIC
# --------------------------
df["neg_high_questions"] = ((df["is_negative"] == 1) & (df["question_count"] > 1)).astype(int)

df["neg_anger_questions"] = (
    (df["is_negative"] == 1) &
    (df["anger_count"] >= 1) &
    (df["question_count"] > 1)
).astype(int)

df["neg_anger_exclaim"] = (
    (df["is_negative"] == 1) &
    (df["anger_count"] >= 1) &
    (df["exclamation_count"] >= 1)
).astype(int)

# --------------------------
# NEGATIVE + CONFUSION LOGIC
# --------------------------
df["neg_confused_longtext"] = (
    (df["is_negative"] == 1) &
    (df["confusion_count"] >= 1) &
    (df["text_len"] > df["text_len"].median())
).astype(int)

# --------------------------
# NEG + MULTIPLE SIGNALS (HEAVY FRUSTRATION)
# --------------------------
df["neg_many_signs"] = (
    (df["is_negative"] == 1) &
    (
        (df["question_count"] > 1).astype(int)
        + (df["exclamation_count"] > 0).astype(int)
        + (df["anger_count"] > 0).astype(int)
        + (df["confusion_count"] > 0).astype(int)
    ) >= 2
).astype(int)

# --------------------------
# MACRO EMOTION FLAGS
# --------------------------
df["neg_angry"] = ((df["is_negative"] == 1) & (df["emotion_macro"] == "Angry")).astype(int)
df["neg_confused"] = ((df["is_negative"] == 1) & (df["emotion_macro"] == "Confused")).astype(int)
df["neg_mixed"] = ((df["is_negative"] == 1) & (df["emotion_macro"] == "Mixed")).astype(int)

df["neutral_but_longtext"] = (
    (df["is_neutral"] == 1) & (df["text_len"] > df["text_len"].median())
).astype(int)

# --------------------------
# INTERACTION COMPLEXITY SCORE
# --------------------------
df["interaction_complexity"] = 0

df.loc[
    (df["text_len"] > df["text_len"].median()) |
    (df["question_count"] > 1) |
    (df["punct_density"] > df["punct_density"].median()),
    "interaction_complexity"
] = 1

df.loc[
    (df["text_len"] > df["text_len"].quantile(0.75)) |
    (df["question_count"] > 2) |
    (df["punct_density"] > df["punct_density"].quantile(0.75)),
    "interaction_complexity"
] = 2

# --------------------------
# RISK LEVEL ENGINEERING
# --------------------------
df["risk_level_basic"] = 0

df.loc[
    (df["is_negative"] == 1) &
    ((df["anger_count"] > 0) | (df["confusion_count"] > 0)),
    "risk_level_basic"
] = 1

df.loc[
    (df["is_negative"] == 1) &
    (df["severity"] > 1.5) &
    (df["interaction_complexity"] == 2),
    "risk_level_basic"
] = 2


In [24]:
core_cols = [
    "Utterance", "clean_text",
    "Sentiment",
    "emotion_macro",
    "intent_macro"
]

# ---- Complaint / urgency / severity ----
signal_cols = [
    "is_negative", "is_positive", "is_neutral",
    "is_complaint", "is_request",
    "is_urgent", "severity",
    "sentiment_conflict"
]

# ---- Emotion micro-signals ----
emotion_cols = [
    "anger_count", "confusion_count", "gratitude_count"
]

# ---- Linguistic features ----
linguistic_cols = [
    "text_len", "word_count", "avg_word_len",
    "question_count", "exclamation_count",
    "punct_density",
    "tb_polarity", "tb_subjectivity",
    "readability_score"
]

# ---- Topic / category information ----
topic_cols = [
    "Topic", "Topic_Name", "fallback_type"
]

# ---- Your engineered logic columns ----
engineered_cols = [
    col for col in df.columns
    if col.startswith("neg_") or col.startswith("complex_")
]

# ---- Everything else (if missed) ----
remaining_cols = [
    col for col in df.columns
    if col not in (core_cols + signal_cols + emotion_cols +
                   linguistic_cols + topic_cols + engineered_cols)
]

# ---- Final ordered list ----
ordered_cols = (
    core_cols +
    signal_cols +
    emotion_cols +
    linguistic_cols +
    topic_cols +
    engineered_cols +
    remaining_cols
)

df = df[ordered_cols]


In [26]:
df.risk_level_basic.value_counts()

risk_level_basic
0    13554
2     1886
1      158
Name: count, dtype: int64

In [28]:
df.to_csv("cci_data_reordered.csv", index=False)

df.columns

Index(['Utterance', 'clean_text', 'Sentiment', 'emotion_macro', 'intent_macro',
       'is_negative', 'is_positive', 'is_neutral', 'is_complaint',
       'is_request', 'is_urgent', 'severity', 'sentiment_conflict',
       'anger_count', 'confusion_count', 'gratitude_count', 'text_len',
       'word_count', 'avg_word_len', 'question_count', 'exclamation_count',
       'punct_density', 'tb_polarity', 'tb_subjectivity', 'readability_score',
       'Topic', 'Topic_Name', 'fallback_type', 'neg_high_questions',
       'neg_anger_questions', 'neg_anger_exclaim', 'neg_confused_longtext',
       'neg_many_signs', 'neg_angry', 'neg_confused', 'neg_mixed',
       'frustration_count', 'neutral_but_longtext', 'interaction_complexity',
       'risk_level_basic'],
      dtype='object')